In [1]:
import torch 
import torchvision
from torch import nn,optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [2]:
data_csv=pd.read_csv('./data.csv',usecols=[1])

In [3]:
data_csv.shape

(145, 1)

In [8]:
data_csv=data_csv.dropna()
dataset=data_csv.apply(lambda x : x/(x.max()-x.min()))
dataset=dataset.values
dataset=dataset.astype('float32')

In [9]:
dataset.shape

(144, 1)

In [10]:
dataX,dataY=[],[]
for i in range(len(dataset)-2):
    dataX.append(dataset[i:i+2])
    dataY.append(dataset[i+2])

In [11]:
dataX[-10:]

[array([[0.8050193 ],
        [0.75482625]], dtype=float32), array([[0.75482625],
        [0.8088803 ]], dtype=float32), array([[0.8088803 ],
        [0.88996136]], dtype=float32), array([[0.88996136],
        [0.9111969 ]], dtype=float32), array([[0.9111969],
        [1.0328186]], dtype=float32), array([[1.0328186],
        [1.2007722]], dtype=float32), array([[1.2007722],
        [1.1698842]], dtype=float32), array([[1.1698842],
        [0.980695 ]], dtype=float32), array([[0.980695  ],
        [0.88996136]], dtype=float32), array([[0.88996136],
        [0.7528958 ]], dtype=float32)]

In [12]:
dataY[-10:]

[array([0.8088803], dtype=float32),
 array([0.88996136], dtype=float32),
 array([0.9111969], dtype=float32),
 array([1.0328186], dtype=float32),
 array([1.2007722], dtype=float32),
 array([1.1698842], dtype=float32),
 array([0.980695], dtype=float32),
 array([0.88996136], dtype=float32),
 array([0.7528958], dtype=float32),
 array([0.8339768], dtype=float32)]

In [13]:
train_size=int(len(dataX)*0.7)
test_size=len(dataX)-train_size
train_X=dataX[:train_size]
train_Y=dataY[:train_size]
test_X=dataX[train_size:]
test_Y=dataY[train_size:]

In [15]:
train_X[0].shape

(2, 1)

In [16]:
train_x=torch.Tensor(train_X).reshape(-1,1,2)
train_y=torch.Tensor(train_Y).reshape(-1,1,1)
test_x=torch.Tensor(test_X).reshape(-1,1,2)
test_y=torch.Tensor(test_Y).reshape(-1,1,1)

In [17]:
train_x.shape

torch.Size([99, 1, 2])

In [18]:
class LSTM_REG(nn.Module):
    def __init__(self,input_size,hidden_size,output_size=1,num_layers=2):
        super().__init__()
        self.rnn=nn.LSTM(input_size,hidden_size,num_layers)
        self.reg=nn.Linear(hidden_size,output_size)
        
    def forward(self,x):
        x,_=self.rnn(x)
        s,b,h=x.shape
        x=x.view(s*b,h)
        x=self.reg(x)
        x=x.view(s,b,-1)
        return x

In [20]:
net=LSTM_REG(2,4)
loss=nn.MSELoss()
optimizer=optim.Adam(net.parameters(),lr=1e-2)
for e in range(1000):
    x=train_x
    y=train_y
    l=loss(net(x),y)
    optimizer.zero_grad()
    l.backward()
    optimizer.step()
    if (e+1) % 100 ==0:
        print('epoch %d,loss : %.5f' % (e+1,l.item()))

epoch 100,loss : 0.00372
epoch 200,loss : 0.00360
epoch 300,loss : 0.00355
epoch 400,loss : 0.00335
epoch 500,loss : 0.00357
epoch 600,loss : 0.00310
epoch 700,loss : 0.00214
epoch 800,loss : 0.00136
epoch 900,loss : 0.00124
epoch 1000,loss : 0.00109


#### add h and c

In [22]:
class LSTM_REG2(nn.Module):
    def __init__(self,input_size,hidden_size,output_size=1,num_layers=2):
        super().__init__()
        self.rnn=nn.LSTM(input_size,hidden_size,num_layers)
        self.reg=nn.Linear(hidden_size,output_size)
        self.state=None
    
    def forward(self,x,state):
        x,self.state=self.rnn(x,state)
        s,b,h=x.shape
        x=x.view(s*b,h)
        x=self.reg(x)
        x=x.view(s,b,-1)
        return x,self.state

In [25]:
net=LSTM_REG2(2,4)
loss=nn.MSELoss()
optimizer=optim.Adam(net.parameters(),lr=1e-2)
state=None
for e in range(2000):
    x=train_x
    y=train_y
    if state is not None:
        state=(state[0].detach(),state[1].detach())
    (y_hat,state)=net(x,state)
    l=loss(y_hat,y)
    optimizer.zero_grad()
    l.backward()
    optimizer.step()
    if (e+1) % 100 ==0:
        print('epoch %d,loss : %.5f' % (e+1,l.item()))
    

epoch 100,loss : 0.01936
epoch 200,loss : 0.01795
epoch 300,loss : 0.00552
epoch 400,loss : 0.00333
epoch 500,loss : 0.00240
epoch 600,loss : 0.00202
epoch 700,loss : 0.00189
epoch 800,loss : 0.00182
epoch 900,loss : 0.00177
epoch 1000,loss : 0.00173
epoch 1100,loss : 0.00169
epoch 1200,loss : 0.00167
epoch 1300,loss : 0.00166
epoch 1400,loss : 0.00165
epoch 1500,loss : 0.00165
epoch 1600,loss : 0.00164
epoch 1700,loss : 0.00164
epoch 1800,loss : 0.00164
epoch 1900,loss : 0.00163
epoch 2000,loss : 0.00163
